In [1]:
import pandas as pd
import numpy as np
import math
import os
import pickle
from collections import defaultdict

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, r2_score

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib as mpl
import matplotlib.font_manager as fm

mpl.rcParams['axes.unicode_minus'] = False

path = 'C:/Windows/Fonts/malgun.ttf'
font_name = fm.FontProperties(fname=path, size=50).get_name()
plt.rc('font', family=font_name)

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima

plt.style.use('seaborn-whitegrid')
%matplotlib inline

import itertools

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
import matplotlib

matplotlib.rcParams['font.family'] ='Malgun Gothic'

matplotlib.rcParams['axes.unicode_minus'] =False

In [4]:
import datetime

import pymysql
import MySQLdb

import FinanceDataReader as fdr


# 추천회사

In [5]:
def company_recomend(date):
    end_point = "chatbot-db.c9x08hbiunuu.ap-northeast-2.rds.amazonaws.com"
    port =3306
    user_name = 'root'
    pw ='123123123'

    conn = pymysql.connect(
        host = end_point,
        user = user_name,
        password = pw,
    #     db = db,
        charset='utf8'

    )
    cursor = conn.cursor()

    sql = 'use Chatbot_DB;'
    cursor.execute(sql)

    sql = 'select * from stock_table;'

    stock_table = pd.read_sql(sql, conn)
    
    
    stock_table['날짜'] = stock_table['날짜'].astype('datetime64')
    stock_df = stock_table[stock_table['날짜'] == date]
    stock_df = stock_df.drop(['BPS', 'PER', 'PBR', 'EPS', 'DIV_', 'DPS', 'quarter','날짜','등락률','name'], axis=1)
    stock_df = stock_df.astype(str).apply(lambda col : col.apply(lambda x: np.nan if x == 'nan' else x))
    stock_df.dropna(inplace=True)

    hoho = defaultdict(list)
    plus_df = pd.DataFrame()
    hoit_df = pd.DataFrame()
    fail_li = []
    code_li = []
    month_li = ['30일','3개월','6개월']
    per_li = ['5per','10per','15per']

    for code in stock_df['code'][:]:
        try:
            df = pd.read_csv(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_재무데이터/머신러닝_train_test/6개월/test/{code}.csv', index_col=0)
            df.drop(['date','등락률','종목명','수익률','quarter','5per','10per','15per'], axis=1, inplace=True)
            df.rename(columns={'법인세차감전 순이익':'법인세차감전_순이익'}, inplace=True)

            ya_df = pd.concat([stock_df[['code']],stock_df[list(df.columns)]], axis=1)
            x_test = ya_df[ya_df.code == code].iloc[0,1:]

            for monmon in month_li[:]:
                for per in per_li[:]:
                        with open(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_모델/{monmon}_{per}/Randomforest/{code}.pkl','rb') as f:
                                rfc = pickle.load(f)

                        pred = rfc.predict([x_test])

                        col = monmon+'_'+per

                        hoho[col].append(pred[0])
            code_li.append(code)

        except:
#             print(code)
            pass

    plus_df = pd.DataFrame({'code':code_li})

    for col,val in hoho.items():
        plus_df[col] = val
    
#     print(plus_df.head())
    
    mon_1 = ['30일','3개월','6개월']
    hoit_li_1 = []
    hoit_li_2 = []
    hoit_com_1 = []
    hoit_com_2 = []
    hoit_com_3 = []
    
    for mon_n in mon_1:
        max_plus_df = plus_df[plus_df[f'{mon_n}_15per']== 1]
        max_plus_df.reset_index(drop=True, inplace=True)

        pre_df = pd.read_csv(f'c:/Users/bitcamp/Desktop/final_data/모델정리/종목별_머신러닝/시연용/rfc_{mon_n}_15.csv', index_col=0)
        pre_df.code = pre_df.code.apply(lambda x:f'{x:06}')

        company_li = []
        com_code_li = []
        for pre in range(len(pre_df)):
            for coco in range(len(max_plus_df)):
                if max_plus_df.iloc[coco, 0] == pre_df.iloc[pre, 0]:
                    company_li.append(pre_df.iloc[pre,1])
                    com_code_li.append(pre_df.iloc[pre,0])
                    
        hoit_li_1.append(mon_n)
#         hoit_li_2.append([company_li[0].strip('[]').strip("''"), company_li[1].strip('[]').strip("''"), company_li[2].strip('[]').strip("''")])
        hoit_com_1.append(com_code_li[0].strip('[]').strip("''"))
        hoit_com_2.append(com_code_li[1].strip('[]').strip("''"))
        hoit_com_3.append(com_code_li[2].strip('[]').strip("''"))
#         hoit = pd.DataFrame({'기간':hoit_li_1, '추천회사':hoit_li_2})
        hoit = pd.DataFrame({'기간':hoit_li_1, '추천회사_1':hoit_com_1, '추천회사_2':hoit_com_2, '추천회사_3':hoit_com_3})
        
    return hoit

In [6]:
hoit = company_recomend('2022-03-02')
hoit

기간  추천회사_1  추천회사_2  추천회사_3
0  30일  092200  005850  032350
1  3개월  004250  161000  020150
2  6개월  010770  014990  018670

In [7]:
hoit = company_recomend('2022-01-03')
hoit

기간  추천회사_1  추천회사_2  추천회사_3
0  30일  013580  042660  032350
1  3개월  084010  002140  004250
2  6개월  014990  018670  003280

In [8]:
hoit = company_recomend('2021-10-01')
hoit

기간  추천회사_1  추천회사_2  추천회사_3
0  30일  092200  005850  032350
1  3개월  004250  161000  020150
2  6개월  010770  014990  018670

# 1개월 코스피 비교

In [5]:
bm_df = fdr.DataReader('092200','2022-03-02','2022-03-31')
bm_1 = round((bm_df.iloc[-1,3] - bm_df.iloc[0,3])/bm_df.iloc[0,3]*100, 2)

kospi_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/fdr_kospi.csv')
kospi = kospi_df[['Date','Close']]
kospi = kospi['2022-03-01'< kospi['Date']]
kospi = kospi[kospi['Date'] <'2022-04-01']

kospi_bm = round((kospi.iloc[-1,1] - kospi.iloc[0,1])/kospi.iloc[0,1]*100, 2)

print('모델 수익률 : ', bm_1)
print('코스피 수익률 : ', kospi_bm)
print('BM대비 수익률 : ', round(bm_1-kospi_bm, 2))

모델 수익률 :  4.07
코스피 수익률 :  2.0
BM대비 수익률 :  2.07


In [6]:
bm_df = fdr.DataReader('005850','2022-03-02','2022-03-31')
bm_1 = round((bm_df.iloc[-1,3] - bm_df.iloc[0,3])/bm_df.iloc[0,3]*100, 2)

kospi_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/fdr_kospi.csv')
kospi = kospi_df[['Date','Close']]
kospi = kospi['2022-03-01'< kospi['Date']]
kospi = kospi[kospi['Date'] <'2022-04-01']

kospi_bm = round((kospi.iloc[-1,1] - kospi.iloc[0,1])/kospi.iloc[0,1]*100, 2)

print('모델 수익률 : ', bm_1)
print('코스피 수익률 : ', kospi_bm)
print('BM대비 수익률 : ', round(bm_1-kospi_bm, 2))

모델 수익률 :  4.65
코스피 수익률 :  2.0
BM대비 수익률 :  2.65


In [9]:
bm_df = fdr.DataReader('032350','2022-03-02','2022-03-31')
bm_1 = round((bm_df.iloc[-1,3] - bm_df.iloc[0,3])/bm_df.iloc[0,3]*100, 2)

kospi_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/fdr_kospi.csv')
kospi = kospi_df[['Date','Close']]
kospi = kospi['2022-03-01'< kospi['Date']]
kospi = kospi[kospi['Date'] <'2022-04-01']

kospi_bm = round((kospi.iloc[-1,1] - kospi.iloc[0,1])/kospi.iloc[0,1]*100, 2)

print('모델 수익률 : ', bm_1)
print('코스피 수익률 : ', kospi_bm)
print('BM대비 수익률 : ', round(bm_1-kospi_bm, 2))

모델 수익률 :  0.28
코스피 수익률 :  2.0
BM대비 수익률 :  -1.72


# 3개월 코스피 비교

In [10]:
bm_df = fdr.DataReader('084010','2022-01-03','2022-03-31')
bm_1 = round((bm_df.iloc[-1,3] - bm_df.iloc[0,3])/bm_df.iloc[0,3]*100, 2)

kospi_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/fdr_kospi.csv')
kospi = kospi_df[['Date','Close']]
kospi = kospi['2022-01-02'< kospi['Date']]
kospi = kospi[kospi['Date'] <'2022-04-01']

kospi_bm = round((kospi.iloc[-1,1] - kospi.iloc[0,1])/kospi.iloc[0,1]*100, 2)

print('모델 수익률 : ', bm_1)
print('코스피 수익률 : ', kospi_bm)
print('BM대비 수익률 : ', round(bm_1-kospi_bm, 2))

모델 수익률 :  29.87
코스피 수익률 :  -7.73
BM대비 수익률 :  37.6


In [11]:
bm_df = fdr.DataReader('002140','2022-01-03','2022-03-31')
bm_1 = round((bm_df.iloc[-1,3] - bm_df.iloc[0,3])/bm_df.iloc[0,3]*100, 2)

kospi_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/fdr_kospi.csv')
kospi = kospi_df[['Date','Close']]
kospi = kospi['2022-01-02'< kospi['Date']]
kospi = kospi[kospi['Date'] <'2022-04-01']

kospi_bm = round((kospi.iloc[-1,1] - kospi.iloc[0,1])/kospi.iloc[0,1]*100, 2)

print('모델 수익률 : ', bm_1)
print('코스피 수익률 : ', kospi_bm)
print('BM대비 수익률 : ', round(bm_1-kospi_bm, 2))

모델 수익률 :  25.21
코스피 수익률 :  -7.73
BM대비 수익률 :  32.94


In [12]:
bm_df = fdr.DataReader('004250','2022-01-03','2022-03-31')
bm_1 = round((bm_df.iloc[-1,3] - bm_df.iloc[0,3])/bm_df.iloc[0,3]*100, 2)

kospi_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/fdr_kospi.csv')
kospi = kospi_df[['Date','Close']]
kospi = kospi['2022-01-03'< kospi['Date']]
kospi = kospi[kospi['Date'] <'2022-04-01']

kospi_bm = round((kospi.iloc[-1,1] - kospi.iloc[0,1])/kospi.iloc[0,1]*100, 2)

print('모델 수익률 : ', bm_1)
print('코스피 수익률 : ', kospi_bm)
print('BM대비 수익률 : ', round(bm_1-kospi_bm, 2))

모델 수익률 :  -15.49
코스피 수익률 :  -7.75
BM대비 수익률 :  -7.74


# 6개월 코스피 비교

In [13]:
bm_df = fdr.DataReader('010770','2021-10-01','2022-03-31')
bm_1 = round((bm_df.iloc[-1,3] - bm_df.iloc[0,3])/bm_df.iloc[0,3]*100, 2)

kospi_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/fdr_kospi.csv')
kospi = kospi_df[['Date','Close']]
kospi = kospi['2021-10-01'< kospi['Date']]
kospi = kospi[kospi['Date'] <'2022-04-01']

kospi_bm = round((kospi.iloc[-1,1] - kospi.iloc[0,1])/kospi.iloc[0,1]*100, 2)

print('모델 수익률 : ', bm_1)
print('코스피 수익률 : ', kospi_bm)
print('BM대비 수익률 : ', round(bm_1-kospi_bm, 2))

모델 수익률 :  -17.69
코스피 수익률 :  -6.9
BM대비 수익률 :  -10.79


In [14]:
bm_df = fdr.DataReader('014990','2021-10-01','2022-03-31')
bm_1 = round((bm_df.iloc[-1,3] - bm_df.iloc[0,3])/bm_df.iloc[0,3]*100, 2)

kospi_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/fdr_kospi.csv')
kospi = kospi_df[['Date','Close']]
kospi = kospi['2021-10-01'< kospi['Date']]
kospi = kospi[kospi['Date'] <'2022-04-01']

kospi_bm = round((kospi.iloc[-1,1] - kospi.iloc[0,1])/kospi.iloc[0,1]*100, 2)

print('모델 수익률 : ', bm_1)
print('코스피 수익률 : ', kospi_bm)
print('BM대비 수익률 : ', round(bm_1-kospi_bm, 2))

모델 수익률 :  -50.42
코스피 수익률 :  -6.9
BM대비 수익률 :  -43.52


In [15]:
bm_df = fdr.DataReader('018670','2021-10-01','2022-03-31')
bm_1 = round((bm_df.iloc[-1,3] - bm_df.iloc[0,3])/bm_df.iloc[0,3]*100, 2)

kospi_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/fdr_kospi.csv')
kospi = kospi_df[['Date','Close']]
kospi = kospi['2021-10-01'< kospi['Date']]
kospi = kospi[kospi['Date'] <'2022-04-01']

kospi_bm = round((kospi.iloc[-1,1] - kospi.iloc[0,1])/kospi.iloc[0,1]*100, 2)

print('모델 수익률 : ', bm_1)
print('코스피 수익률 : ', kospi_bm)
print('BM대비 수익률 : ', round(bm_1-kospi_bm, 2))

모델 수익률 :  -25.57
코스피 수익률 :  -6.9
BM대비 수익률 :  -18.67


# 2022-01-02

In [5]:
def company_recomend(date):
    end_point = "chatbot-db.c9x08hbiunuu.ap-northeast-2.rds.amazonaws.com"
    port =3306
    user_name = 'root'
    pw ='123123123'

    conn = pymysql.connect(
        host = end_point,
        user = user_name,
        password = pw,
    #     db = db,
        charset='utf8'

    )
    cursor = conn.cursor()

    sql = 'use Chatbot_DB;'
    cursor.execute(sql)

    sql = 'select * from stock_table;'

    stock_table = pd.read_sql(sql, conn)
    
    
    stock_table['날짜'] = stock_table['날짜'].astype('datetime64')
    stock_df = stock_table[stock_table['날짜'] == date]
    stock_df = stock_df.drop(['BPS', 'PER', 'PBR', 'EPS', 'DIV_', 'DPS', 'quarter','날짜','등락률','name'], axis=1)
    stock_df = stock_df.astype(str).apply(lambda col : col.apply(lambda x: np.nan if x == 'nan' else x))
    stock_df.dropna(inplace=True)

    hoho = defaultdict(list)
    plus_df = pd.DataFrame()
    hoit_df = pd.DataFrame()
    fail_li = []
    code_li = []
    month_li = ['30일','3개월','6개월']
    per_li = ['5per','10per','15per']

    for code in stock_df['code'][:]:
        try:
            df = pd.read_csv(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_재무데이터/머신러닝_train_test/6개월/test/{code}.csv', index_col=0)
            df.drop(['date','등락률','종목명','수익률','quarter','5per','10per','15per'], axis=1, inplace=True)
            df.rename(columns={'법인세차감전 순이익':'법인세차감전_순이익'}, inplace=True)

            ya_df = pd.concat([stock_df[['code']],stock_df[list(df.columns)]], axis=1)
            x_test = ya_df[ya_df.code == code].iloc[0,1:]

            for monmon in month_li[:]:
                for per in per_li[:]:
                        with open(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_모델/{monmon}_{per}/Randomforest/{code}.pkl','rb') as f:
                                rfc = pickle.load(f)

                        pred = rfc.predict([x_test])

                        col = monmon+'_'+per

                        hoho[col].append(pred[0])
            code_li.append(code)

        except:
#             print(code)
            pass

    plus_df = pd.DataFrame({'code':code_li})

    for col,val in hoho.items():
        plus_df[col] = val
    
#     print(plus_df.head())
    
    mon_1 = ['30일','3개월','6개월']
    hoit_li_1 = []
    hoit_li_2 = []
    hoit_com_1 = []
    hoit_com_2 = []
    hoit_com_3 = []
    
    for mon_n in mon_1:
        max_plus_df = plus_df[plus_df[f'{mon_n}_15per']== 1]
        max_plus_df.reset_index(drop=True, inplace=True)

        pre_df = pd.read_csv(f'c:/Users/bitcamp/Desktop/final_data/모델정리/종목별_머신러닝/시연용/rfc_{mon_n}_15.csv', index_col=0)
        pre_df.code = pre_df.code.apply(lambda x:f'{x:06}')

        company_li = []
        com_code_li = []
        for pre in range(len(pre_df)):
            for coco in range(len(max_plus_df)):
                if max_plus_df.iloc[coco, 0] == pre_df.iloc[pre, 0]:
                    company_li.append(pre_df.iloc[pre,1])
                    com_code_li.append(pre_df.iloc[pre,0])
                    
        hoit_li_1.append(mon_n)
#         hoit_li_2.append([company_li[0].strip('[]').strip("''"), company_li[1].strip('[]').strip("''"), company_li[2].strip('[]').strip("''")])
        hoit_com_1.append(com_code_li[0].strip('[]').strip("''"))
        hoit_com_2.append(com_code_li[1].strip('[]').strip("''"))
        hoit_com_3.append(com_code_li[2].strip('[]').strip("''"))
#         hoit = pd.DataFrame({'기간':hoit_li_1, '추천회사':hoit_li_2})
        hoit = pd.DataFrame({'기간':hoit_li_1, '추천회사_1':hoit_com_1, '추천회사_2':hoit_com_2, '추천회사_3':hoit_com_3})
        
    return hoit

In [6]:
hoit = company_recomend('2022-01-03')
hoit

기간  추천회사_1  추천회사_2  추천회사_3
0  30일  013580  042660  032350
1  3개월  084010  002140  004250
2  6개월  014990  018670  003280

## 1개월 비교

In [16]:
bm_df = fdr.DataReader('013580','2022-01-02','2022-01-31')
bm_1 = round((bm_df.iloc[-1,3] - bm_df.iloc[0,3])/bm_df.iloc[0,3]*100, 2)

kospi_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/fdr_kospi.csv')
kospi = kospi_df[['Date','Close']]
kospi = kospi['2022-01-01'< kospi['Date']]
kospi = kospi[kospi['Date'] <'2022-02-01']

kospi_bm = round((kospi.iloc[-1,1] - kospi.iloc[0,1])/kospi.iloc[0,1]*100, 2)

print('모델 수익률 : ', bm_1)
print('코스피 수익률 : ', kospi_bm)
print('BM대비 수익률 : ', round(bm_1-kospi_bm, 2))

모델 수익률 :  -6.5
코스피 수익률 :  -10.89
BM대비 수익률 :  4.39


In [17]:
bm_df = fdr.DataReader('042660','2022-01-02','2022-01-31')
bm_1 = round((bm_df.iloc[-1,3] - bm_df.iloc[0,3])/bm_df.iloc[0,3]*100, 2)

kospi_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/fdr_kospi.csv')
kospi = kospi_df[['Date','Close']]
kospi = kospi['2022-01-01'< kospi['Date']]
kospi = kospi[kospi['Date'] <'2022-02-01']

kospi_bm = round((kospi.iloc[-1,1] - kospi.iloc[0,1])/kospi.iloc[0,1]*100, 2)

print('모델 수익률 : ', bm_1)
print('코스피 수익률 : ', kospi_bm)
print('BM대비 수익률 : ', round(bm_1-kospi_bm, 2))

모델 수익률 :  -13.94
코스피 수익률 :  -10.89
BM대비 수익률 :  -3.05


In [18]:
bm_df = fdr.DataReader('032350','2022-01-02','2022-01-31')
bm_1 = round((bm_df.iloc[-1,3] - bm_df.iloc[0,3])/bm_df.iloc[0,3]*100, 2)

kospi_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/fdr_kospi.csv')
kospi = kospi_df[['Date','Close']]
kospi = kospi['2022-01-01'< kospi['Date']]
kospi = kospi[kospi['Date'] <'2022-02-01']

kospi_bm = round((kospi.iloc[-1,1] - kospi.iloc[0,1])/kospi.iloc[0,1]*100, 2)

print('모델 수익률 : ', bm_1)
print('코스피 수익률 : ', kospi_bm)
print('BM대비 수익률 : ', round(bm_1-kospi_bm, 2))

모델 수익률 :  -17.18
코스피 수익률 :  -10.89
BM대비 수익률 :  -6.29


## 3개월 비교

In [19]:
bm_df = fdr.DataReader('084010','2022-01-02','2022-03-31')
bm_3 = round((bm_df.iloc[-1,3] - bm_df.iloc[0,3])/bm_df.iloc[0,3]*100, 2)

kospi_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/fdr_kospi.csv')
kospi = kospi_df[['Date','Close']]
kospi = kospi['2022-01-01'< kospi['Date']]
kospi = kospi[kospi['Date'] <'2022-04-01']

kospi_bm = round((kospi.iloc[-1,1] - kospi.iloc[0,1])/kospi.iloc[0,1]*100, 2)

print('모델 수익률 : ', bm_3)
print('코스피 수익률 : ', kospi_bm)
print('BM대비 수익률 : ', round(bm_3-kospi_bm, 2))

모델 수익률 :  29.87
코스피 수익률 :  -7.73
BM대비 수익률 :  37.6


In [20]:
bm_df = fdr.DataReader('002140','2022-01-02','2022-03-31')
bm_3 = round((bm_df.iloc[-1,3] - bm_df.iloc[0,3])/bm_df.iloc[0,3]*100, 2)

kospi_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/fdr_kospi.csv')
kospi = kospi_df[['Date','Close']]
kospi = kospi['2022-01-01'< kospi['Date']]
kospi = kospi[kospi['Date'] <'2022-04-01']

kospi_bm = round((kospi.iloc[-1,1] - kospi.iloc[0,1])/kospi.iloc[0,1]*100, 2)

print('모델 수익률 : ', bm_3)
print('코스피 수익률 : ', kospi_bm)
print('BM대비 수익률 : ', round(bm_3-kospi_bm, 2))

모델 수익률 :  25.21
코스피 수익률 :  -7.73
BM대비 수익률 :  32.94


In [21]:
bm_df = fdr.DataReader('004250','2022-01-02','2022-03-31')
bm_3 = round((bm_df.iloc[-1,3] - bm_df.iloc[0,3])/bm_df.iloc[0,3]*100, 2)

kospi_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/fdr_kospi.csv')
kospi = kospi_df[['Date','Close']]
kospi = kospi['2022-01-01'< kospi['Date']]
kospi = kospi[kospi['Date'] <'2022-04-01']

kospi_bm = round((kospi.iloc[-1,1] - kospi.iloc[0,1])/kospi.iloc[0,1]*100, 2)

print('모델 수익률 : ', bm_3)
print('코스피 수익률 : ', kospi_bm)
print('BM대비 수익률 : ', round(bm_3-kospi_bm, 2))

모델 수익률 :  -15.49
코스피 수익률 :  -7.73
BM대비 수익률 :  -7.76


## 6개월 비교

In [23]:
bm_df = fdr.DataReader('014990','2022-01-02')
bm_6 = round((bm_df.iloc[-1,3] - bm_df.iloc[0,3])/bm_df.iloc[0,3]*100, 2)

kospi_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/fdr_kospi.csv')
kospi = kospi_df[['Date','Close']]
kospi = kospi['2022-01-01'< kospi['Date']]
# kospi = kospi[kospi['Date'] <'2022-04-01']

kospi_bm = round((kospi.iloc[-1,1] - kospi.iloc[0,1])/kospi.iloc[0,1]*100, 2)

print('모델 수익률 : ', bm_6)
print('코스피 수익률 : ', kospi_bm)
print('BM대비 수익률 : ', round(bm_6-kospi_bm, 2))

모델 수익률 :  -37.88
코스피 수익률 :  -22.42
BM대비 수익률 :  -15.46


In [24]:
bm_df = fdr.DataReader('018670','2022-01-02')
bm_6 = round((bm_df.iloc[-1,3] - bm_df.iloc[0,3])/bm_df.iloc[0,3]*100, 2)

kospi_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/fdr_kospi.csv')
kospi = kospi_df[['Date','Close']]
kospi = kospi['2022-01-01'< kospi['Date']]
# kospi = kospi[kospi['Date'] <'2022-04-01']

kospi_bm = round((kospi.iloc[-1,1] - kospi.iloc[0,1])/kospi.iloc[0,1]*100, 2)

print('모델 수익률 : ', bm_6)
print('코스피 수익률 : ', kospi_bm)
print('BM대비 수익률 : ', round(bm_6-kospi_bm, 2))

모델 수익률 :  -2.01
코스피 수익률 :  -22.42
BM대비 수익률 :  20.41


In [25]:
bm_df = fdr.DataReader('003280','2022-01-02')
bm_6 = round((bm_df.iloc[-1,3] - bm_df.iloc[0,3])/bm_df.iloc[0,3]*100, 2)

kospi_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/fdr_kospi.csv')
kospi = kospi_df[['Date','Close']]
kospi = kospi['2022-01-01'< kospi['Date']]
# kospi = kospi[kospi['Date'] <'2022-04-01']

kospi_bm = round((kospi.iloc[-1,1] - kospi.iloc[0,1])/kospi.iloc[0,1]*100, 2)

print('모델 수익률 : ', bm_6)
print('코스피 수익률 : ', kospi_bm)
print('BM대비 수익률 : ', round(bm_6-kospi_bm, 2))

모델 수익률 :  -24.04
코스피 수익률 :  -22.42
BM대비 수익률 :  -1.62
